<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#question-1" data-toc-modified-id="question-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>question 1</a></span></li><li><span><a href="#question-2" data-toc-modified-id="question-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>question 2</a></span></li></ul></div>

In [1]:
import numpy as np

In [2]:
digits = set(list("0123456789"))

In [3]:
with open("input.txt") as f:
    data = [ list(line.strip()) for line in f.readlines() ]

In [4]:
ndata = np.array(data, dtype=str)

In [5]:
for irow in range(3):
    print("".join(ndata[irow,:100]))

........440...............418..643.....438......740.261......................................727....
...............338.............-........*.......*.......34&.$........@.....&742.....................
..................*369.....334.......624..749....533........690...894...........466......../....&...


In [6]:
ijmax = 140

In [78]:
def A(n):
    if n<0:
        return 0
    if n>ijmax-1:
        return ijmax-1
    else:
        return n

## question 1

In [13]:
def check_symbol(ndata, irow, j, ndigits):
    around = ndata[slice(A(irow-1),A(irow+2)),slice(A(j-1),A(j+ndigits+1))]
    #print("around", around.flatten())
    return any( not char in "1234567890." for char in around.flatten() )


def run_row(irow):
    row = ndata[irow,:]
    new_numbers = []
    j = 0
    while True:
        s = row[j]
        if s in digits:
            ndigits = 1
            while True:
                if j+ndigits<=ijmax-1 and row[j+ndigits] in digits:
                    ndigits += 1
                else:
                    break
            number = int("".join(row[j:j+ndigits]))
            
            # check if there is a symbol around the number. If yes, add to list
            if check_symbol(ndata, irow, j, ndigits):
                new_numbers.append(number)
            j += ndigits
        else:
            j += 1
        if j>=ijmax:
            break
    return new_numbers

In [14]:
N = []
for i in range(140):
    new_numbers = run_row(i)
    N.extend(new_numbers)

In [15]:
sum(N)

532331

## question 2

In [79]:
ndata = np.pad(ndata, pad_width=3, mode='constant', constant_values='.')
ijmax = 146

In [80]:
def look_left_right(around, right=False, i=1):
    number = []
    j = 3
    while True:
        if right:
            j += 1
        else:
            j -= 1
            if j<0:
                break
        try:
            c = around[i,j]
        except IndexError:
            break
        if c in digits:
            number.append(c)
        else:
            break
    if right:
        return number
    else:
        return number[::-1]

In [81]:
def look_above_below(around, below=False):
    if below:
        i = 2
    else:
        i = 0
    try:
        c = around[i,3]
    except IndexError:
        if below:
            return []
        else:
            raise
    if c in digits:
        number = []
        leftdigits = look_left_right(around, i=i)
        rightdigits = look_left_right(around, right=True, i=i)
        number.extend(leftdigits)
        number.append(c)
        number.extend(rightdigits)
        return [number]
    else:
        numbers = []
        if leftdigits := look_left_right(around, i=i):
            numbers.append(leftdigits)
        if rightdigits := look_left_right(around, right=True, i=i):
            numbers.append(rightdigits)
        return numbers

In [82]:
def look_for_numbers(around):
    """ star is at [1,3]
       [['.' '.' '4' '3' '8' '.' '.']
        ['.' '.' '.' '*' '.' '.' '.']
        ['6' '2' '4' '.' '.' '7' '4']]"""
    i,j = 1,3
    N = []
    # look left:
    n = look_left_right(around)
    if n: N.append(n)
    # look right:
    n = look_left_right(around, right=True)
    if n: N.append(n)
    # look above:
    ns = look_above_below(around)
    N.extend(ns)
    # look below:
    ns = look_above_below(around, below=True)
    N.extend(ns)
    #print("N:", N)
    return N

In [86]:
def find_gear(irow):
    row = ndata[irow,:]
    new_numbers = []
    j = 0
    while True:
        s = row[j]
        if s=="*":
            around = ndata[slice(irow-1,irow+2),slice(j-3,j+3+1)]
            print(around)
            N = look_for_numbers(around)
            if len(N)==2:
                Nints = list(map(lambda x: int("".join(x)), N))
                print("found N", Nints)
                new_numbers.append(np.prod(Nints))
        j += 1
        if j>=ijmax:
            break
    return new_numbers

In [95]:
all_numbers = []
for i in range(148):
    new_numbers = find_gear(i)
    all_numbers.extend(new_numbers)
total = sum(all_numbers)
print(total)

[['.' '.' '4' '3' '8' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['6' '2' '4' '.' '.' '7' '4']]
found N [438, 624]
[['.' '.' '.' '7' '4' '0' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['.' '.' '.' '.' '5' '3' '3']]
found N [740, 533]
[['.' '8' '7' '0' '.' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['.' '.' '.' '.' '1' '.' '.']]
found N [870, 1]
[['3' '3' '8' '.' '.' '.' '.']
 ['.' '.' '.' '*' '3' '6' '9']
 ['.' '.' '.' '.' '.' '.' '.']]
found N [369, 338]
[['.' '.' '.' '.' '3' '6' '7']
 ['.' '.' '.' '*' '.' '.' '.']
 ['.' '.' '.' '.' '8' '8' '6']]
found N [367, 886]
[['.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['4' '5' '7' '.' '.' '8' '6']]
[['.' '.' '5' '6' '5' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['.' '.' '.' '7' '3' '9' '.']]
found N [565, 739]
[['.' '4' '5' '6' '.' '3' '5']
 ['.' '.' '.' '*' '.' '.' '.']
 ['6' '.' '.' '.' '1' '3' '1']]
found N [456, 131]
[['7' '.' '.' '.' '.' '8' '7']
 ['.' '.' '.' '*' '.' '.' '.']
 ['.' '.' '.' '.' '5' '6' '1']]
[['4' '7' '3' '.' '.' '

In [93]:
find_gear(147)

[['.' '8' '9' '6' '.' '/' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['4' '2' '3' '.' '.' '.' '.']]
found N [896, 423]
[['.' '/' '.' '.' '.' '.' '.']
 ['.' '.' '.' '*' '.' '.' '.']
 ['7' '4' '2' '.' '3' '6' '7']]
found N [742, 367]


[379008, 272314]